In [ ]:
# %load ../../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from datetime import datetime
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC, NWTG
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
import zcode.math as zmath
import zcode.plot as zplot

In [ ]:
log.setLevel(log.DEBUG)

In [ ]:
log.level, log.DEBUG, log.INFO

In [ ]:
SHAPE = 20
TIME = 1.0 * GYR

sam = holo.sam.Semi_Analytic_Model(shape=SHAPE)
hard = holo.hardening.Fixed_Time.from_sam(sam, TIME, interpolate_norm=True)

In [ ]:
STEPS = 500

# () start from the hardening model's initial separation
rmax = hard._sepa_init
# (M,) end at the ISCO
rmin = utils.rad_isco(sam.mtot)
# rmin = hard._TIME_TOTAL_RMIN * np.ones_like(sam.mtot)
# Choose steps for each binary, log-spaced between rmin and rmax
extr = np.log10([rmax * np.ones_like(rmin), rmin])
rads = np.linspace(0.0, 1.0, STEPS)[np.newaxis, :]
# (M, X)
rads = extr[0][:, np.newaxis] + (extr[1] - extr[0])[:, np.newaxis] * rads
rads = 10.0 ** rads
# (M, Q, Z, X)
mt, mr, rz, rads = np.broadcast_arrays(
    sam.mtot[:, np.newaxis, np.newaxis, np.newaxis],
    sam.mrat[np.newaxis, :, np.newaxis, np.newaxis],
    sam.redz[np.newaxis, np.newaxis, :, np.newaxis],
    rads[:, np.newaxis, np.newaxis, :]
)
# (X, M*Q*Z)
mt, mr, rz, rads = [mm.reshape(-1, STEPS).T for mm in [mt, mr, rz, rads]]
# (X, M*Q*Z) --- `Fixed_Time.dadt` will only accept this shape
dadt = hard.dadt(mt, mr, rads)
# Integrate (inverse) hardening rates to calculate total lifetime to each separation
times_evo = -utils.trapz_loglog(-1.0 / dadt, rads, axis=0, cumsum=True)


In [ ]:
tt = times_evo[-1, :]/GYR
fig, ax = plot.figax(scale='lin')
print(utils.stats(tt))
kale.dist1d(tt, density=True)
plt.show()

In [ ]:
bads = (tt > 1.8)
print(tt[bads])
print(utils.frac_str(bads))
bads = np.where(bads)[0]
print(bads)
print()
print(mt[0][bads]/MSOL)
print(mr[0][bads])
print(rz[0][bads])

In [ ]:
rads.shape

In [ ]:
fig, ax = plot.figax()
tw = ax.twinx()
tw.set(yscale='log')
# for bb in bads:
# for bb in np.random.choice(rads.shape[1], 10, replace=False):
for bb in [98]:
    print(bb)
    xx = rads[:, bb]/PC
    ax.plot(xx[1:], times_evo[:, bb]/GYR, label=bb, alpha=0.5)
    tau = - rads[:, bb]/dadt[:, bb]
    tw.plot(xx, tau/YR, label=bb, ls='--', alpha=0.5)

ax.legend()    
plt.show()